# What are GANs?
GANs, or [Generative Adversarial Networks](https://arxiv.org/abs/1406.2661), are a framework for estimating generative models. Two models are trained simultaneously by an adversarial process: a Generator, which is responsible for generating data (say, images), and a Discriminator, which is responsible for estimating the probability that an image was drawn from the training data (the image is real), or was produced by the Generator (the image is fake). During training, the Generator becomes progressively better at generating images, until the Discriminator is no longer able to distinguish real images from fake. 
​
![alt text](https://github.com/margaretmz/tensorflow/blob/margaret-dcgan/tensorflow/contrib/eager/python/examples/generative_examples/gans_diagram.png?raw=1)
​
We will demonstrate this process end-to-end on MNIST. Below is an animation that shows a series of images produced by the Generator as it was trained for 50 epochs. Overtime, the generated images become increasingly difficult to distinguish from the training set.
​
To learn more about GANs, we recommend MIT's [Intro to Deep Learning](http://introtodeeplearning.com/) course, which includes a lecture on Deep Generative Models ([video](https://youtu.be/JVb54xhEw6Y) | [slides](http://introtodeeplearning.com/materials/2018_6S191_Lecture4.pdf)). Now, let's head to the code!
​
![sample output](https://tensorflow.org/images/gan/dcgan.gif)

## The Generator Model

The generator is responsible for creating convincing images that are good enough to fool the discriminator. The network architecture for the generator consists of [Conv2DTranspose](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose) (Upsampling) layers. We start with a fully connected layer and upsample the image two times in order to reach the desired image size of 28x28x1. We increase the width and height, and reduce the depth as we move through the layers in the network. We use [Leaky ReLU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LeakyReLU) activation for each layer except for the last one where we use a tanh activation.

## The Discriminator model

The discriminator is responsible for distinguishing fake images from real images. It's similar to a regular CNN-based image classifier.



# How to train a GAN?

Follow the tips/tricks given in https://github.com/soumith/ganhacks and in [medium](https://medium.com/@utk.is.here/keep-calm-and-train-a-gan-pitfalls-and-tips-on-training-generative-adversarial-networks-edd529764aa9) to find out the best design and strategies to train your GAN architecture.

# Importing Libraries


In [1]:
import numpy as np
import glob
import imageio
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.ndimage
import cv2

from scipy import misc
from keras.models import Model
from keras.layers import *
from keras import backend as K
from keras.engine import *
from keras.optimizers import Adam
from keras.datasets import mnist
from skimage.transform import resize
from google.colab import drive

Using TensorFlow backend.


# Mounting Google Drive (gdrive)

This ipnb save the training process on you google drive account. You drive must contain the following directory structure in "My Drive": /DCGAN/MNIST

In [2]:
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


# Auxiliary Functions

In [0]:
def imread(f):
    x = misc.imread(f)
    x = x[center_height:center_height + width, :]
    x = misc.imresize(x, (img_dim, img_dim))

    if len(x.shape) == 2:
        x = np.expand_dims(x, axis=2)

    return x.astype(np.float32) / 255 * 2 - 1

In [0]:
#imgs = glob.glob('/path/to/images/*.png')

In [0]:
def data_generator(batch_size=64):
    X = []
    while True:
        np.random.shuffle(imgs)
        for f in imgs:
            X.append(imread(f))
            if len(X) == batch_size:
                X = np.array(X)
                yield X
                X = []

In [0]:
def mnist_generator(X_data, batch_size=64):
  idx = np.random.randint(0, X_data.shape[0], batch_size)
  imgs = X_data[idx]
  return imgs

 # DCGAN Architecture

In [0]:
class DCGAN():

    def __init__(self, imdim=128, imchannels=1, im_minsze=4, latent_dim=128, filter_sze=5, batch_size=32,
                 disc_filters=64, disc_dropout=0.25, disc_bn_momentum=0.8, disc_add_noise=False,
                 gen_bn_momentum=0.8, gen_act = 'relu',
                 iter_disc=1, iter_gen=1, save_interval=100, model_name='dc_gan', dataset_name='mnist', dataset_path='', dataset_out=''):
        # Input shape
        self.imdim = imdim
        self.imrows = self.imdim
        self.imcols = self.imdim
        self.imchannels = imchannels
        self.img_min_sze = im_minsze
        self.img_shape = (self.imrows, self.imcols, self.imchannels)
        self.batch_size = batch_size
        self.latent_dim = latent_dim
        self.filter_sze = filter_sze
        self.disc_filters = disc_filters
        self.disc_depth = 3
        self.disc_dropout = disc_dropout
        self.disc_bn_momentum = disc_bn_momentum
        self.disc_add_noise = disc_add_noise
        self.gen_act = gen_act
        self.gen_filters = 64
        self.gen_bn_momentum = gen_bn_momentum
        self.optimizer = Adam(2e-4, 0.5)
        self.iter_disc = iter_disc  # number of discriminator network updates per adversarial training step
        self.iter_gen = iter_gen  # number of generative network updates per adversarial training step
        self.save_interval = save_interval

        outDir = 'pix{}_ksze{}_zdim{}'.format(self.imdim, self.filter_sze, self.latent_dim)

        # Configure data loader
        self.dataset_name = dataset_name
        self.model_name = model_name
        self.dataset_path = dataset_path
        self.logs_dir = os.path.join(self.dataset_path + '/logs/')
        self.preds_dir = os.path.join(self.dataset_path +'/preds/')
        self.dataset_out = dataset_out #os.path.join(dataset_out + get_timestamp())
        self.model_out = os.path.join(self.dataset_out + '/model_{}/'.format(outDir))
        self.samples_out = os.path.join(self.dataset_out +'/samples_{}/'.format(outDir))

        if not os.path.exists(self.dataset_out):
            os.makedirs(self.dataset_out)

        if not os.path.exists(self.samples_out):
            os.makedirs(self.samples_out)

        if not os.path.exists(self.model_out):
            os.makedirs(self.model_out)

        # Build the discriminator
        self.discriminator = self.build_discriminator()

        # Build the generator
        self.generator = self.build_generator()

        # Combined model
        x_in = ... # define input here
        z_in = ... # define input here

        # Configuring Discriminator
        self.generator.trainable = False

        x_real = x_in
        x_fake = ... # generate fake image

        x_real_score = ... # get discriminator score on real image
        x_fake_score = ... # get discriminator score on fake image

        self.discriminator_combined = Model(...) # create combined model here.

        d_loss = ... # define discriminator loss here
        self.discriminator_combined.add_loss(d_loss)
        self.discriminator_combined.compile(optimizer=self.optimizer)

        # Configuring Generator
        self.generator.trainable = True
        self.discriminator.trainable = False

        x_real = x_in
        x_fake = ... # generate fake image here

        x_real_score = ... # get discriminator score on real image
        x_fake_score = ... # get discriminator score on fake image

        self.generator_combined = Model(...) # create generator model here

        g_loss = ... # define generator loss here
        self.generator_combined.add_loss(g_loss)
        self.generator_combined.compile(optimizer=self.optimizer)

        # Displaying final models
        self.discriminator_combined.summary()
        self.generator_combined.summary()


    def build_discriminator(self):
      # create discriminator here.
      # use sigmoid activation function for output
      
      model = Sequential()

      model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dropout(0.25))
      
      model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
      model.add(ZeroPadding2D(padding=((0,1),(0,1))))
      model.add(BatchNormalization(momentum=0.8))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dropout(0.25))
      
      model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dropout(0.25))
      
      model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dropout(0.25))
      
      model.add(Flatten())
      model.add(Dense(1, activation='sigmoid'))
      model.summary()

      img = Input(shape=self.img_shape)
      validity = model(img)

      return Model(img, validity)


    def build_generator(self):

      # create generator here.
      # use tanh activation function for output
      
      model = Sequential()

      model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
      model.add(Reshape((7, 7, 128)))
      model.add(UpSampling2D())
      model.add(Conv2D(128, kernel_size=3, padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation("relu"))
      model.add(UpSampling2D())
      model.add(Conv2D(64, kernel_size=3, padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation("relu"))
      model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
      
      model.add(Activation("tanh"))
      model.summary()

      noise = Input(shape=(self.latent_dim,))
      img = model(noise)

      return Model(noise, img)

    def train(self, epochs):

        batch_size = self.batch_size
        
        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        
        X_train_sze = []
        for img in X_train:
            X_train_sze.append(np.expand_dims(cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC),axis=2))
        X_train_sze = np.array(X_train_sze)
        print(X_train.shape)
        print(X_train_sze.shape)
        X_train = X_train_sze
        
        #img_generator = data_generator(batch_size)

        for epoch in range(epochs):

            for ii in range(self.iter_disc):
                # Sample noise and generate a batch of new images
                z_sample = ...
                d_loss = ... # train discriminator here and get loss

            for ii in range(self.iter_gen):
                z_sample = ...
                g_loss = ... # train generator here and get loss

            if epoch % 10 == 0:
                print('epoch: {}, d_loss: {}, g_loss: {}'.format(epoch, d_loss, g_loss))

            if epoch % self.save_interval == 0:
                out_file = '{}/epoch_{}.png'.format(self.samples_out, epoch)
                self.save_images_noborder(out_file)
                #self.save_images_wborder(out_file)
                #self.save_model(self.model_out)


    def save_images_noborder(self, path):
        n = 5
        imdim = self.imdim

        figure = np.zeros((imdim * n, imdim * n, 3))
        for ii in range(n):
            for jj in range(n):
                z_sample = np.random.randn(1, self.latent_dim)
                x_sample = self.generator.predict(z_sample)
                digit = x_sample[0]
                figure[ii * imdim:(ii + 1) * imdim,
                jj * imdim:(jj + 1) * imdim] = digit
        figure = (figure + 1) / 2 * 255
        figure = np.round(figure, 0).astype(int)
        imageio.imwrite(path, figure)
        
        
    def save_images_wborder(self, path):
        r, c = 5, 5
        z_sample = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(z_sample)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(path)
        plt.close()


    def save_model(self, path):

        def save(model, model_name, path=self.model_out):
            model_path = "{}/{}.json".format(path, model_name)
            weights_path = "{}/{}_weights.hdf5".format(path, model_name)
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator_combined, "dcgan_generator_combined", path)
        save(self.discriminator_combined, "dcgan_discriminator_combined", path)
        save(self.generator, "dcgan_generator", path)
        save(self.discriminator, "dcgan_discriminator", path)

## Run

In [8]:
#dcgan = DCGAN(imdim=28, im_minsze=7, latent_dim=100, filter_sze=3, dataset_out='/gdrive/My Drive/data/ucsp-dcgan/mnist/')
dcgan = DCGAN(imdim=32, im_minsze=4, latent_dim=100, filter_sze=5, dataset_out='/gdrive/My Drive/data/ucsp-dcgan/mnist/')
dcgan.train(epochs=10001)

AttributeError: ignored